In [1]:
# coding=utf8
import pymysql
from scipy import stats
import requests,json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
import random 
import re
import pandas as pd
from io import StringIO
import math as m
import statsmodels.formula.api

# def get_data(sql, format='csv_header '):
#     """
#     根据输入的SQL语句，返回数数科技查询结果
#     :param sql: SQL语句
#     :param format: 默认为'csv-header', 可选择的还有json
#     :return: 返回DataFrame
#     """
#     headers = {
#         'Content-Type': 'application/x-www-form-urlencoded',
#     }

#     data = {
#         'sql': sql,
#         'format': format
#     }
#     url = 'http://101.32.192.62:8992/querySql?token=bjMIEMvVgCYH3ngtjTw1hngzuKpdqSkI1GoO1A0NkJQdubjrmHmYBGCH8HRE20tU'
#     r = requests.post(url, headers=headers, data=data)
#     data = StringIO(r.text)  # 将返回数据转化为StringIO对象

#     json_head = json.loads(data.readline())  # 获取头数据，并转化为字典
#     column_name = json_head['data']['headers']
#     column_name = [re.findall('\w.*', k)[0] for k in column_name]  # 去除字段名中的不规范字符

#     # 提取行数据，并对格式进行转化
#     result = [json.loads(line) for line in data.readlines()]

#     # 生成DataFrame
#     df = pd.DataFrame(result, columns=column_name).convert_dtypes()

#     return df

# # table name: ta.v_event_6
# "#event_name", "#account_id", "channel","media_source","#country_code", "app_version"

# if __name__ == '__main__':
#     sql = """
#     SELECT "$part_event","$part_date","#user_id","#event_name","#event_time","#account_id","#distinct_id","#server_time","channel","platform","#zone_offset","#city","#ip","carid","#lib","totalgamelength","install_time","#province","ailevel","deviceid","version","level",
#     "carlevel","state","carjumptimers","finalrewardamount","systemarchitecture","totalgametimesum","totaltimes","failfinishtype","rwtype","loadtime","aggregatelevelfinishtimes","required_enginelevel","gaslevel","required_gaslevel","bonuslevel","enginelevel","finalreward_diamond","aggregatediamondamount","finalreward_coin","aggregatecoinamount","partsid","tyreid","racetype","computeshadersupport" FROM ta.v_event_6 WHERE "$part_date" = '2021-04-06' 
#     """
#     get_data(sql)


def rename_table(name_list, df_data):
    for (index, item) in enumerate(name_list):
        df_data[item] = df_data[index]     
    df_data = df_data.drop([i for i in range(len(name_list))], axis =1)
    return df_data

def list_to_frame(my_df):
    df_aux = my_df.to_frame()
    df_aux = df_aux.reset_index()
    return df_aux

def check_factor_level(factor, my_df):
    drop = my_df.drop_duplicates(subset = [factor], keep = 'first')
    return list(drop[factor])

def str_to_date(obj):
    return datetime.strptime(obj, '%m/%d/%Y')

def date_to_str(obj):
    return obj.strftime('%m/%d/%Y')

def sort_date(date):
    aux_date = list(map(str_to_date, date))
    aux_date.sort()
    final = list(map(date_to_str, aux_date))
    return final

def list_to_frame(my_df):
    df_aux = my_df.to_frame()
    df_aux = df_aux.reset_index()
    return df_aux

def cleaning(file_path, start_date, end_date):
    data = pd.read_csv(file_path)
    data['count'] = 1
    aux = data[data['#event_name'] == 'mpmf_ad_reward'].groupby(['#account_id', '$part_date', 'app_version'])['count'].sum()
    df = list_to_frame(aux)
    start_date = datetime.strptime(start_date,'%m/%d/%Y')
    end_date = datetime.strptime(end_date,'%m/%d/%Y')
    df['datetime'] = df['$part_date'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y'))
    df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date) ]
    cleaned_df = df.groupby(['app_version','#account_id']).sum().reset_index()
    return cleaned_df

def datelist(start_date, end_date):
    # beginDate, endDate是形如‘20210409’的字符串或datetime格式
    date_l=[datetime.strftime(x,'%Y/%m/%d') for x in list(pd.date_range(start=start_date, end=end_date))]
    return date_l

pd.set_option('display.max_rows', 2000)



# Get data frame

# file_path: "20210408_024723_00382_teus3.csv"
# ins_date: "2021/1/31"
# date: "2021/1/31"

# date:要计算到的天数

def data_clean_date(file_path, ins_date, date):  
    
    data = pd.read_csv(file_path)
    data['count'] = 1
    aux = data[data['#event_name'] == 'install'].groupby(['#account_id','$part_date', 'app_version'])['count'].sum()
    df = list_to_frame(aux)
    df['ins_date'] = df['$part_date']
    df_ins = df.drop(['$part_date', 'count'], axis = 1)
    df_date = df_ins[(df_ins['ins_date'] == ins_date)]
    df_date['date'] = date
    
    re = data[data['#event_name'] == 'mpmf_ad_reward'].groupby(['#account_id', '$part_date', 'app_version'])['count'].sum()     
    df_re = list_to_frame(re)
    df_re['date'] = df_re['$part_date']
    df_re = df_re.drop(['$part_date'], axis = 1)
    
    inte = data[data['#event_name'] == 'mpmf_ad_Inter'].groupby(['#account_id', '$part_date', 'app_version'])['count'].sum()     
    df_in = list_to_frame(inte)
    df_in['date'] = df_in['$part_date']
    df_in = df_in.drop(['$part_date'], axis = 1)
    
    df_redate = pd.merge(df_date, df_re, how = 'left', on = ['#account_id', 'date', 'app_version'])
    df_redate['count'] = df_redate['count'].where(df_redate['count'].notnull(), 0)
    df_redate['reward'] = df_redate['count']
    df_redate = df_redate.drop(['count'], axis = 1)
    
    df_indate = pd.merge(df_date, df_in, how = 'left', on = ['#account_id', 'date', 'app_version'])
    df_indate['count'] = df_indate['count'].where(df_indate['count'].notnull(), 0)
    df_indate['inter'] = df_indate['count']
    df_indate = df_indate.drop(['count'], axis = 1)
    
    df_final = pd.merge(df_redate, df_indate, how = 'left', on = ['#account_id', 'app_version','ins_date','date'])
    df_final['total'] = df_final['reward'] + df_final['inter']
 
    return df_final 


def specific_merge(df_date1, df_date2):
    df_date1 = df_date1.drop(['app_version', 'ins_date'], axis = 1)
    df_acudate = pd.merge(df_date1, df_date2, how = 'left', on = ['#account_id'])
    df_acudate['reward'] = df_acudate['reward_x'] + df_acudate['reward_y']
    df_acudate['inter'] = df_acudate['inter_x'] + df_acudate['inter_y']
    df_acudate['total'] = df_acudate['total_x'] + df_acudate['total_y']
    df_acudate['date'] = df_acudate['date_x']
    df_acudate = df_acudate.drop(['reward_x', 'reward_y', 'inter_x', 'inter_y','total_x', 'total_y','date_x', 'date_y'], axis = 1)
    return df_acudate


def data_clean_acumulate_date(file_path, ins_date, date_interval):
    
    final = data_clean_date(file_path, ins_date, date_interval[0])
    if len(date_interval) > 1: 
        for i in date_interval[1:]:
            df_aux = data_clean_date(file_path, ins_date, i)
            final = specific_merge(final, df_aux)
        final['start_date'] = date_interval[0]
        final['end_date'] = date_interval[len(date_interval)-1]
        final = final.drop(['date'], axis = 1)
        return final
    else: 
        final['start_date'] = date_interval[0]
        final['end_date'] = date_interval[len(date_interval)-1]
        final = final.drop(['date'], axis = 1)
        return final 



def data_cleaning_final(path, ins_date, date_interval):
    df_aux = pd.DataFrame()
    for i in range(len(date_interval)):
        df = data_clean_acumulate_date(path, ins_date, date_interval[0:(i + 1)])
        df_aux = pd.concat([df_aux, df])
    df_aux = df_aux.reset_index(drop = True)
    return df_aux


def trimmed_sample_left(colname, df, trim_level):
    final = pd.DataFrame()
    date_in = list(df.drop_duplicates(subset = ['end_date'], keep = 'first')['end_date'])
    for i in date_in:
        aux_1 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.3')].sort_values(by =[colname], ascending=True)
        aux_2 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.4')].sort_values(by =[colname], ascending=True)
        aux_1 = aux_1.iloc[0:(m.ceil(aux_1.shape[0]*trim_level))]
        aux_2 = aux_2.iloc[0:(m.ceil(aux_1.shape[0]*trim_level))]
        aux_3 = pd.concat([aux_1, aux_2])
        final = pd.concat([aux_3, final])
    final = final.reset_index(drop = True)
    return final 


# helpe = trimmed_sample('reward',df, 0.1)
def trimmed_sample_right(colname, df, trim_level):
    final = pd.DataFrame()
    date_in = list(df.drop_duplicates(subset = ['end_date'], keep = 'first')['end_date'])
    for i in date_in:
        aux_1 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.3')].sort_values(by =[colname], ascending=True)
        aux_2 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.4')].sort_values(by =[colname], ascending=True)
        aux_1 = aux_1.iloc[(m.ceil(aux_1.shape[0]*(1-trim_level)) + 1):]
        aux_2 = aux_2.iloc[(m.ceil(aux_2.shape[0]*(1-trim_level)) + 1):]
        aux_3 = pd.concat([aux_1, aux_2])
        final = pd.concat([aux_3, final])
    final = final.reset_index(drop = True)
    return final 

#helpe = trimmed_sample_left_right('reward',df, 0.1)
def trimmed_sample_left_right(colname, df, trim_level):
    final = pd.DataFrame()
    date_in = list(df.drop_duplicates(subset = ['end_date'], keep = 'first')['end_date'])
    for i in date_in:
        aux_1 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.3')].sort_values(by =[colname], ascending=True)
        aux_2 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.4')].sort_values(by =[colname], ascending=True)
        aux_3 = aux_1.iloc[(m.ceil(aux_1.shape[0]*(1-trim_level)) + 1):]
        aux_4 = aux_2.iloc[(m.ceil(aux_2.shape[0]*(1-trim_level)) + 1):]
        aux_5 = aux_1.iloc[0:(m.ceil(aux_1.shape[0]*trim_level))]
        aux_6 = aux_2.iloc[0:(m.ceil(aux_2.shape[0]*trim_level))]
        aux_7 = pd.concat([aux_3, aux_4])
        aux_8 = pd.concat([aux_5, aux_6])
        aux_9 = pd.concat([aux_7, aux_8])
        final = pd.concat([aux_9, final])
    final = final.reset_index(drop = True)
    return final 


def winsorized_sample(colname, df, trim_level):
    final = pd.DataFrame()
    date_in = list(df.drop_duplicates(subset = ['end_date'], keep = 'first')['end_date'])
    for i in date_in:
        aux_1 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.3')].sort_values(by =[colname], ascending=True)
        aux_2 = df[(df['end_date'] == i) & (df['app_version'] == '1.1.4')].sort_values(by =[colname], ascending=True)
        aux_3 = aux_1.iloc[(m.ceil(aux_1.shape[0]*trim_level)):(m.ceil(aux_1.shape[0]*(1-trim_level)))]
        aux_4 = aux_2.iloc[(m.ceil(aux_1.shape[0]*trim_level)):(m.ceil(aux_1.shape[0]*(1-trim_level)))]
        aux_5 = pd.concat([aux_3, aux_4])
        final = pd.concat([aux_5, final])
    final = final.reset_index(drop = True)
    return final 



# def get_csv():
#     date_interval = datelist('20210131', '20210210')
#     for i in range(len(date_interval)):
#     df = data_clean_acumulate_date('20210408_024723_00382_teus3.csv', '2021/1/31', date_interval[0:(i + 1)])
#     d0.to_csv("113114%s.csv"%date_interval[i])


# Experiment_design 数据处理
# WOR_data = pd.read_csv('mpmf_mean(ad_reward)_20210308-20210314.csv')
# WOR = for_experiment_design(WOR_data, 2)
# WOR.to_csv('mpmf_人均_ad_reward_20210308-20210314.csv')

# date_list = ['1/31/2021', '2/1/2021', '2/2/2021', '2/3/2021', '2/4/2021', '2/5/2021']

# for i in date_list:
#     obj_data = cleaning('20210331_113424_01352_teus3.csv', '1/31/2021', i)
#     obj_data['count_num'] = 1
#     print(obj_data.groupby(['app_version']).sum())

# def count(data, num):
#     count = 0 
#     for i in data:
#         if i > num:
#             count = count + 1
#     return print(num, count)

# for i in range(1, 200):
#     count(ver113, i)

# mean113, mean114 = np.mean(d0[d0['app_version'] == '1.1.3']['count'].tolist()), np.mean(d0[d0['app_version'] == '1.1.4']['count'].tolist())
# var113, var114 = np.var(d0[d0['app_version'] == '1.1.3']['count'].tolist()), np.var(d0[d0['app_version'] == '1.1.4']['count'].tolist())
# qver113, qver114 = len(ver113)/(len(ver113) - 1) * var113, len(ver114)/(len(ver114) - 1) * var114
# len(ver113), len(ver114),mean113, mean114, qver113, qver114

# ver113 = d0[d0['app_version'] == '1.1.3']['count']
# ver114 = d0[d0['app_version'] == '1.1.4']['count']
# opt = dict(bins=30, histtype='stepfilled', edgecolor='none')
# plt.axis([0, 200, 0, 2000])
# plt.hist(ver113.to_numpy(), **opt)
# plt.hist(ver114.to_numpy(), **opt)

# pip install statsmodels


In [185]:
# WOR_data = pd.read_csv('20210331_113424_01352_teus3.csv')
# WOR_data = pd.read_csv('20210331_065708_00759_teus3.csv')
# WOR_data.shape, WOR_data.columns
# d0 = cleaning('20210331_113424_01352_teus3.csv', '1/31/2021', '1/31/2021')
# d0.to_csv("113114.csv")
# obj_data[obj_data['app_version'] == '1.1.3']
# v = ver113.to_numpy()
# (v - np.mean(v))/np.var(v)**(1/2)
# (1 - np.mean(v))/np.var(v)**(1/2)
# (2 - np.mean(v))/np.var(v)**(1/2)

# df[df['count'] > 1]
# pd.set_option('display.max_columns', None)
# data[(data['#account_id'] == '1612288316822-6842426706215486083') & (data['#event_name'] == 'install')]
# data[(data['#account_id'] == '1612288316822-6842426706215486083')]

# date_interval = ['2021/1/31', '2021/2/1','2021/2/2','2021/2/3','2021/2/4','2021/2/5','2021/2/6','2021/2/7','2021/2/8','2021/2/9','2021/2/10']
# df.to_csv("113114.csv")
# final = pd.DataFrame()
# date_interval = ['4/7/2021', '4/8/2021', '4/9/2021', '4/10/2021', '4/11/2021','4/12/2021','4/13/2021']
# for i in date_interval: 
#     aux = data_cleaning_final('20210414_033707_00348_gn3qg.csv', i, date_interval)
#     final = pd.concat([aux, final])
# final = final.reset_index(drop = True)  
# final.to_csv("124125")

In [2]:
df = pd.read_csv('20210414_033707_00348_gn3qg.csv')
date_interval = ['2021-04-07','2021-04-08','2021-04-09','2021-04-10','2021-04-11','2021-04-12','2021-04-13']
df = data_cleaning_final('20210414_033707_00348_gn3qg.csv','2021-04-07',date_interval)
df.to_csv('124125.csv')

C:\Users\chenq\anaconda3_new\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-2-66b677e2d038>:3: DtypeWarning: Columns (10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  df = data_cleaning_final('20210414_033707_00348_gn3qg.csv','2021-04-07',date_interval)
<ipython-input-1-176e339a4fd9>:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_date['date'] = date


In [3]:
df

,#account_id,app_version,ins_date,reward,inter,total,start_date,end_date
0,1616635998640-9042719963125327220,1.2.4,2021-04-07,0.0,0.0,0.0,2021-04-07,2021-04-07
1,1617667022457-168525562445343737,1.2.4,2021-04-07,0.0,0.0,0.0,2021-04-07,2021-04-07
2,1617681886683-3723507210924036161,1.2.4,2021-04-07,0.0,0.0,0.0,2021-04-07,2021-04-07
3,1617720015736-9099722131373122173,1.2.4,2021-04-07,0.0,0.0,0.0,2021-04-07,2021-04-07
4,1617724802200-805334560740813243,1.2.5,2021-04-07,1.0,3.0,4.0,2021-04-07,2021-04-07
...,...,...,...,...,...,...,...,...
10369,1617810844031-6991908176145820379,1.2.5,2021-04-07,0.0,3.0,3.0,2021-04-07,2021-04-13
10370,1617810867082-7223207295123513573,1.2.4,2021-04-07,5.0,28.0,33.0,2021-04-07,2021-04-13
10371,1617811066333-1342338812460706216,1.2.5,2021-04-07,0.0,2.0,2.0,2021-04-07,2021-04-13
10372,1617812674165-7552195953642867155,1.2.5,2021-04-07,0.0,6.0,6.0,2021-04-07,2021-04-13


In [7]:
test = df[df['end_date'] == '2021-04-07']
v124 = test[test['app_version'] == '1.2.4']['total']
v125 = test[test['app_version'] == '1.2.5']['total']

In [8]:
stats.ttest_ind(v124,v125, alternative = 'less')

Ttest_indResult(statistic=-0.7670067089542971, pvalue=0.22159992968823977)

In [7]:
df.to_csv('124125.csv')

In [6]:
date_interval = ['2021-04-07','2021-04-08','2021-04-09','2021-04-10','2021-04-11','2021-04-12','2021-04-13']
for i in date_interval:
    print(i)
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.4')]['total'].mean()) 
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.5')]['total'].mean()) 

2021-04-07
5.349112426035503
5.8885400313971745
2021-04-08
7.549112426035503
7.841444270015699
2021-04-09
8.50059171597633
9.10204081632653
2021-04-10
9.246153846153845
10.180533751962324
2021-04-11
9.53491124260355
10.864992150706437
2021-04-12
9.925443786982248
11.232339089481947
2021-04-13
10.111242603550297
11.690737833594977


In [132]:
date_interval = ['2021-04-07','2021-04-08','2021-04-09','2021-04-10','2021-04-11','2021-04-12','2021-04-13']
for i in date_interval:
    print(i)
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.4')]['reward'].mean()) 
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.5')]['reward'].mean()) 

2021-04-07
2.4615384615384617
2.5494505494505493
2021-04-08
3.6828402366863906
3.6671899529042387
2021-04-09
4.106508875739645
4.472527472527473
2021-04-10
4.502958579881657
5.075353218210361
2021-04-11
4.621301775147929
5.489795918367347
2021-04-12
4.772781065088758
5.665620094191523
2021-04-13
4.861538461538461
5.863422291993721


In [133]:
date_interval = ['2021-04-07','2021-04-08','2021-04-09','2021-04-10','2021-04-11','2021-04-12','2021-04-13']
for i in date_interval:
    print(i)
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.4')]['inter'].mean()) 
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.5')]['inter'].mean()) 

2021-04-07
2.8875739644970415
3.339089481946625
2021-04-08
3.8662721893491123
4.17425431711146
2021-04-09
4.3940828402366865
4.629513343799058
2021-04-10
4.743195266272189
5.1051805337519625
2021-04-11
4.913609467455621
5.375196232339089
2021-04-12
5.152662721893491
5.566718995290424
2021-04-13
5.249704142011835
5.827315541601256


In [125]:
date_interval = ['2021-04-08','2021-04-09','2021-04-10','2021-04-11','2021-04-12','2021-04-13']
for i in date_interval:
    print(i)
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.4')]['total'].mean()) 
    print(df[(df['end_date'] == i) & (df['app_version'] == '1.2.5')]['total'].mean()) 

2021-04-08
5.257178526841448
5.0566318926974665
2021-04-09
6.6729088639200995
6.906110283159464
2021-04-10
7.298377028714107
7.786885245901639
2021-04-11
7.742821473158552
8.72876304023845
2021-04-12
8.067415730337078
9.594634873323399
2021-04-13
8.540574282147317
10.070044709388972


In [136]:
print(df[(df['end_date'] == '2021-04-08') & (df['app_version'] == '1.2.4')]['reward'].sum())
print(df[(df['end_date'] == '2021-04-08') & (df['app_version'] == '1.2.4')]['inter'].sum()) 

print(df[(df['end_date'] == '2021-04-08') & (df['app_version'] == '1.2.5')]['reward'].sum())
print(df[(df['end_date'] == '2021-04-08') & (df['app_version'] == '1.2.5')]['inter'].sum()) 

3112.0
3267.0
2336.0
2659.0


In [ ]:
t124 = df[(df['end_date'] == '2021-04-07') & (df['app_version'] == '1.2.4')]['total'].mean()
t125 = df[(df['end_date'] == '2021-04-07') & (df['app_version'] == '1.2.5')]['total'].mean()
stats.ttest_ind(t124, t125)

## Power of function for non-parametric test

In [10]:
date_interval = check_factor_level('$part_date',df)
final = data_cleaning_final('20210414_033707_00348_gn3qg.csv', date_interval)

In [28]:
df['count'] = 1
aux = df[df['#event_name'] == 'install'].groupby(['#account_id','$part_date', 'app_version'])['count'].sum()
df = list_to_frame(aux)
df['ins_date'] = df['$part_date']
df_ins = df.drop(['$part_date', 'count'], axis = 1)
df_ins

,#account_id,app_version,ins_date
0,1616310537153-4096451041333916560,1.2.4,2021-04-13
1,1616595508775-1436936675628725227,1.2.4,2021-04-11
2,1616632939573-937128715024629693,1.2.4,2021-04-09
3,1616635998640-9042719963125327220,1.2.4,2021-04-07
4,1616774716492-4881919259140384884,1.2.5,2021-04-09
...,...,...,...
9779,1618329335521-7143476158041775056,1.2.5,2021-04-13
9780,1618329376623-5465467047830775450,1.2.4,2021-04-13
9781,1618329418719-3648959904582759357,1.2.5,2021-04-13
9782,1618329445540-2754650690079065775,1.2.5,2021-04-13


In [75]:
helpe = trimmed_sample_left('reward',df, 0.1)
helpe.to_csv("113114_left.csv")

In [73]:
helpe = winsorized_sample('reward',df, 0.1)
helpe.to_csv("113114_winsorized.csv")

## Experimental design

## One-way ANOVA with fixed model (balanced data):

### Idea about variability(no variance): 
$$\sum_{i=1}^{N} (x_i - \overline{x})^2$$

$$(y_1, x_1), (y_2,x_2)...(y_n, x_n)$$

### one fixed effect model 

$$ y_{ij} = \mu + \alpha_i + \epsilon_{ij}
\begin{cases}
i = 1, ..., a = 2 \\
j = 1, 2, ...,n
\end{cases}
$$

$$ 
\sum_{i=1}^{2} \alpha_i = 0
$$

$$
\epsilon_{ij} \sim {\sf N}(0, \sigma^2)
$$


### Basic Statistics: 

$$
SS_{T} = \sum_{i=1}^{2} \sum_{j=1}^{n} (y_{ij} - \overline{y}_{..})^2
\\
SS_{TR} = \sum_{i=1}^{2} \sum_{j=1}^{n} (\overline{y}_{i.} - \overline{y}_{..})^2
\\
SS_{E} = \sum_{i=1}^{2} \sum_{j=1}^{n} (y_{ij} - \overline{y}_{i.})^2
\\
SS_{T} = SS_{TR} + SS_{E}
\\
R^2 = \frac{SS_{TR}}{SS_{T}}
$$

### Degrees of freedom:

$$
DF(SS_T) = N - 1, N = 2*n \\
\\
DF(SS_{TR}) = a - 1 = 2 - 1 = 1 \\ 
\\
DF(SS_{E}) = N - a = 2n - 2 \\
\\
MS_{TR} = \frac{SS_{TR}}{a - 1} = SS_{TR} \\
\\
MS_{E} = \frac{SS_E}{N - a} = \frac{SS_E}{2n - 2}
$$

### Hypotesis:
$$ H_0: \alpha_1 = \alpha_2 \\
   H_1: \alpha_1 \not= \alpha_2 
$$

$\alpha = P(type$ $I$ $error) = P(reject H_0|H_0$ $is$ $true)$

$\beta = P(type$ $II$ $error) = P(fail$ $to$ $reject$ $H_0|H_0$ $is$ $false)$

$Power = 1 -\beta = P(reject H_0|H_0$ $is$ $false)$

### Theory:

$$
\frac{SS_E}{\sigma^2} \sim \chi^2, (Fisher) \\
\\
E(MS_E) = \sigma^2, E(\frac{SS_E}{\sigma^2}) = N - a \\
\\
SS_{TR} \sim \chi^2_{1}\mid H_0 \\
\\
E(MS_{TR}) = \sigma^2 + n\frac{\sum_{i = 1}^{2} \alpha^2_{i}}{2 - 1}, (Cochran) \\
\\
\frac{MS_{TR}}{MS_{E}} \sim F_{2-1, 2n - 2} \\
\\
CI: \left\{ \frac{MS_{TR}}{MS_E} > F_{2-1, 2n - 2,\alpha} \right\}
$$

### Estimation:

$$
\hat{\mu} = \overline{y}_{..} \\
\\
\hat{\alpha}_i = \overline{y}_{i.} - \overline{y}_{..} \\ 
\\
\hat{\mu + \alpha_i} = \overline{y}_{i.} \\
\\
\hat{y}_{ij} = \hat{\mu} + \hat{\alpha}_{i} \\
\\
\epsilon_{ij} = y_{ij} - \hat{y}_{ij} = y_{ij} - \hat{\mu} - \hat{\alpha}_i \\
\\
SS_{TR} = \sum_{i=1}^{a} \sum_{j=1}^{n} \hat{\alpha}_i^2 = n\left( \hat{\alpha}_1^2+\hat{\alpha}_2^2+...+\hat{\alpha}_a^2 \right) \\
\\
SS_E = \sum_{i=1}^{a} \sum_{j=1}^{n} (y_{ij} - \hat{y}_{ij})^2
$$

### We can get: 
$$ 
y_{11},..., y_{1n} \sim N(\mu + \alpha_1, \sigma^2) \\
\\
y_{21},..., y_{2n} \sim N(\mu + \alpha_2, \sigma^2) 
$$

### Unbiased estimate(mean, variance):
$$\overline{x} = \frac{1}{n} \sum_{i=1}{n} x_i \\
s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \overline{x})^2
$$

### one way ANOVA： 

 VS   | SS   | DF     | MS   |     F    |
----- | ---- |------- |------|----------|
 TR   | SSTR | a - 1  | MSTR | MSTR/MSE |
Error | SSE  | N - a  | MSE  |          |   
Total | SST  | N - 1  |      |          |

### Unbalanced data:

$$ y_{ij} = \mu + \alpha_i + \epsilon_{ij}
\begin{cases}
i = 1, ..., a = 2 \\
j = 1, 2, ...,n_i
\end{cases}
$$

$$
N = \sum_{i=1}^{a} n_i \\
\\
\sum_{i=1}^{2} \alpha_i = 0 \\
\\
\epsilon_{ij} \sim {\sf N}(0, \sigma^2) \\
\\
SS_{T} = \sum_{i=1}^{2} \sum_{j=1}^{n_i} (y_{ij} - \overline{y}_{..})^2 \\
\\
SS_{TR} = \sum_{i=1}^{2} \sum_{j=1}^{n_i} (\overline{y}_{i.} - \overline{y}_{..})^2 \\
\\
SS_{E} = \sum_{i=1}^{2} \sum_{j=1}^{n_i} (y_{ij} - \overline{y}_{i.})^2
$$

### There are two advantages in choosing a balanced design. First, the test statistic is relatively insensitive to small departures from the assumption of equal variances for the a treatments if the sample sizes are equal. This is not the case for unequal sample sizes. Second, the power of the test is maximized if the samples are of equal size.

## Validation: analysis of residual

### 1. Normality: qq-plot, Kolmogorov-Smirnov test, shapiro-wilk test
### 2. Independent: Durbin-watson statistic 
$$
d = \frac{\sum_{t=2}^{T}(e_t - e_{t-1})^2}{\sum_{t=1}^{T} e_t^2}
$$


### T is the number of observations, since d is approximately equal to $2(1 - \hat{\rho})$, where $\hat{\rho}$, in general, 

$$
\rho = \frac{cov(x, y)}{\sqrt{var(x)}\sqrt{var(y)}}
$$

###  $d = 2$ indicates no autocorrelation, The value of $d$ always lies between $0$ and $4$. If the Durbin–Watson statistic is substantially less than $2$, there is evidence of positive serial correlation. As a rough rule of thumb, if Durbin–Watson is less than $1.0$, there may be cause for alarm. Small values of $d$ indicate successive error terms are positively correlated. If $d > 2$, successive error terms are negatively correlated.

### 3. Homoscedasticity: Levene test 
$$
H_0: \sigma_1 = ... = \sigma_a
\\
H_1: \exists i \not= j, \sigma_i \not= \sigma_j
$$

$D_{ij} = |e_{ij}|$

$\overline{D}_{i.} = \sigma_i$
### Apply experiment design. 
$$
CI: \left\{ \frac{MS_{TR}(D)}{MS_E(D)} > F_{a-1, N - 2,\alpha} \right\}
$$

### Practically, log transformation for 1 and 3. In general, Using box-cox transformation. 

### Box-Cox transformation 
### $I$ groups with observations, $i = 1,2,...,I$, $\mu_i$ and $\sigma_i^2$ for each group. 
$$ 
\exists c,\alpha \in \mathbb{R}|\sigma_i = c\mu_i^\alpha \\
\\
\lambda = 1 - \alpha
$$


### In practice, maximum likelihood method to estimate $c$ and $\alpha$
$$
L(\lambda;y^{\lambda}_1,...,y^{\lambda}_n = -\frac{n}{2}lnSSE(\lambda)
$$

### Formula for transformation:
$$
y^{\lambda} = 
\begin{cases}
\frac{y^{\lambda} - 1}{\lambda\tilde{y}^{\lambda -1}}, \lambda \not= 0 \\
\tilde{y}lny, \lambda = 0 
\end{cases}
$$
### $\tilde{y} = \sqrt[n]{y_i***y_n}$(geometric mean)

### Normality can not handle, using nonparametric test. 

## 1 quantitative + 1 cualitative
### 2 samples dependent:  wilcoxon test
### 2 samples independent : wilcoxon mann whitney test 
### k samples dependent: Friedman test 
### k samples independent: Kruskal wallis test 
## 2 cualitative 
### dependent: McNemar test 
### independent: chi-squared test 
## 2 quantitative
### kendall test, Spearman test 

## wilcoxon mann whitney test

$$
H_0: F_x = F_y \\
H_1: F_x \not= F_y\\
\\
(x_1,...,x_n), (y_1,...,y_m)\\
n < m \\
\Psi_{ij} = \begin{cases}
1, y_j > x_i \\
0, y_j < x_i \\
\end{cases}
i = 1...n, j=1...m \\
U_Y = \sum_{i=1}^{n} \sum_{j=1}^{m} \Psi_{ij} \\
\forall y_j > x_i, U_Y = nm\\
\forall y_j < x_i, U_Y = 0\\ 
U_Y = [0, nm] \\
\Psi_{ij}^* = \begin{cases}
1, y_j < x_i \\
0, y_j > x_i \\
\end{cases}
i = 1...n, j=1...m \\
U_X = \sum_{i=1}^{n} \sum_{j=1}^{m} \Psi_{ij}^* \\
U_X = nm - U_Y\\
x_i = y_i, \Psi_{ij} = \frac{1}{2}, \Psi_{ij}^* = \frac{1}{2}
U = min(U_X, U_Y)\\
n,m < 30, table\\
n, m > 30, U \sim N\left( \frac{nm}{2}, \sqrt{\frac{nm(n+m+1)}{12}} \right) \\
CI: \left\{ U \le k_1 \right\} \cup \left\{ U \ge k_2 \right\}
$$

## Specific comparision

### Least significant difference (Fisher)
$$
H_0: \mu + \alpha_i = \mu + \alpha_j \forall i \not= j \\
H_1: \mu + \alpha_i \not= \mu + \alpha_j
\\
CI(LSD):|\overline{y_{i.}}-\overline{y_{j.}}| > t_{N-a, \frac{\alpha}{2}}\sqrt{MS_E\left(\frac{1}{n}+\frac{1}{n}\right)}
$$

### Bonferonni
$$
H_0: \mu + \alpha_i = \mu + \alpha_j \forall i \not= j \\ 
H_1: \mu + \alpha_i \not= \mu + \alpha_j
\\
CI(LSD):|\overline{y_{i.}}-\overline{y_{j.}}| > t_{N-a, \frac{\alpha_{\beta}}{2}}\sqrt{MS_E\left(\frac{1}{n}+\frac{1}{n}\right)}\\
\alpha_{\beta} = \frac{\alpha}{\binom{a}{2}}
$$

### Honestly significant difference (Tukey)

$$
(x_1,...,x_n) \sim N(0, \sigma^2) \\
Studentized : \frac{X}{\hat{\sigma}} \\
q(l,n) = \frac{max{\{x_1,..,x_n\}} - min{\{x_1,...,x_n\}}}{\hat{\sigma}} \\
\frac{l\hat{\sigma}^2}{\sigma^2} \sim \chi_l^2 \\
\overline{y}_{1.} \le \overline{y}_{2.} \le...\overline{y}_{a.}\\
H_0: \mu + \alpha_1 = \mu + \alpha_a \\ 
H_1: \mu + \alpha_1 \not= \mu + \alpha_a\\
\frac{\overline{y}_{a.}-\overline{y}_{1.}}{\sqrt{\frac{MS_E}{n}}}|H_0 \sim q_\alpha(N - a, a) \\
CI(HSD): \overline{y}_{a.}-\overline{y}_{1.} > q_\alpha(N - a, a)\sqrt{\frac{MS_E}{n}} \\
H_0: \mu + \alpha_i = \mu + \alpha_j  \\ 
H_1: \mu + \alpha_i \not= \mu + \alpha_j \\
CI(HSD): |\overline{y}_{i.}-\overline{y}_{j.}| > q_{\alpha}(N - a, a)\sqrt{\frac{MS_E}{n}} 
$$

### Newman-Keuls Method 
$$
H_0: \mu + \alpha_i = \mu + \alpha_j  \\ 
H_1: \mu + \alpha_i \not= \mu + \alpha_j \\
CI = \left\{ |\overline{y}_{i.}-\overline{y}_{j.}| > q_\alpha(N-a,|pos_{(i)} - pos_{(j)} + 1|)\sqrt{\frac{MS_E}{n}} \right\}
$$

### Duncan Method 
$$
H_0: \mu + \alpha_i = \mu + \alpha_j  \\ 
H_1: \mu + \alpha_i \not= \mu + \alpha_j \\
CI = \left\{ |\overline{y}_{i.}-\overline{y}_{j.}| > r_\alpha(N-a,|pos_{(i)} - pos_{(j)} + 1|)\sqrt{\frac{MS_E}{n}} \right\}
$$

### Dunnet method 

$$
H_0: \mu + \alpha_i = \mu + \alpha_a  \\ 
H_1: \mu + \alpha_i \not= \mu + \alpha_a \\
CI = \left\{ |\overline{y}_{i.}-\overline{y}_{a.}| > d_\alpha(N-a)\sqrt{MS_E\left(\frac{1}{n}+\frac{1}{n}\right)} \right\}
$$


### Repeated Measures

### Robust inference(One-Way Robust testing) 

$$
H_0: \mu_{t1} = \mu_{t2} =...=\mu_{tJ}  
$$
### Using trimmed means rather than means (20% trimming level) cuts off 20% at the lower end and 20% the higher end of the distribution
### 

$$
d = 
$$

# RF模型和AB test内容说明

## 一份完整的数据挖掘报告分为以下四个部分：
## 1. 版本变更内容说明
## 2. 数据可视化
## 3. 统计建模
## &ensp; 3.1 RF模型
## &ensp; 3.2 AB test模型
## 4. 结论报告

$$
\\
$$
### 1. 版本变更说明
$$
\\
$$
<font size=4>
此部分说明游戏的版本变动内容以及依据这些变动确定参数矩阵(metric，包含直接受影响的参数矩阵 + 激励和插页观看次数矩阵)，同时描述样本的基础信息。 
</font>
$$
\\
$$

### 2. 数据可视化
$$
\\
$$
<font size=4>
此部分分为两块，第一块是根据两个版本的游戏的基础数据信息（留存率，广告点击次数之类的重要指标）可视化，此处会用到最为基础的柱状图，饼图，漏斗模型图等。第二块是服务于RF，聚类模型和AB test的可视化，包括RF模型的用户散点图，基于参数矩阵的频率分布柱状图，箱型图，交互作用图等。
</font>
    
$$
\\
$$
### 3. 统计建模
$$
\\
$$
<font size=4>
第3部分主要是对所选择的模型的大概描述和介绍，比如，目前做AB test所选择的 ANOVA 和 wilcoxon mann whitney test。给出具体结果table。考虑具体建模过程可以appendix的形式放在最后以供验收和校正使用，该部分主要是用于每次统计模型的判断将和经验主义所做出的判断一起与实际情况进行比较，从而不断修正模型，供数据中台内部使用。   
</font>    

### 4. 结论报告
$$
\\
$$
<font size=4>
第四部分是基于第三部分的统计结果做出结论，版本之间是否有显著性差异，直接受影响矩阵是否影响了最终的目标矩阵，比如点击率的升高是否会直接使激励的点击次数提升。  
</font>  

## 整体分析思路：
## RF模型 & 聚类

<font size=4>
1. RF模型的目的是给所有玩游戏的用户做分类找出“有问题但有救用户”，设为目标用户。该类用户是在游戏时长或者点击次数某一维度存在问题的用户，找出该部分用户是进一步做归因分析的前提，归因分析是后续结合经验调整版本的依据。用户在游戏时长或者点击次数都很低的不会被作为目标用户是因为该类用户在游戏中的行为过少无法分析，这部分用户只能凭借经验主义去优化。同时，游戏时长和点击次数都高的“优秀用户”也会被监控，确保版本的变化不会影响该部分用户。</font>

$$
\\
$$

<font size=4>
2. 通过RF模型的分类找出目标群体后，使用K-means方法或clustering方法给目标群体进行聚类分群，使用不同的指标一起作为分类标准，最终把目标群体按照他们游戏内的不同行为分成几组，而后通过每组内群体的游戏行为的不同推出此类用户游戏时长或点击次数不足的原因。  
</font>

### 再结合AB版本二次聚类，找出A，B版本中靠近的群设定为AA test的sample 

## 版本比较 ( AB test ) 

<font size=4>
1. 版本变动后分别在A，B两个版本做AA test，使用2中的sample，此处使用repeated measure model， 理论上会获得两个结论，A版本是否有显著性差异和B版本是否有显著性差异。理想情况是，A版本并无明显显著性差异，B版本有显著性差异。 
</font>


$$
\\
$$
<font size=4>
2. 根据版本的变动所造成的影响选择复合或单个metric做AB test，此处为A，B版本间比较，使用robust ANOVA 和 non parametric test，按不同天数得到结论，理想情况是，A和B版本在robust ANOVA和non parametric上存在显著性差异。 
</font>

$$
\\
$$
<font size=4>
3. 优化：缩短test时间周期，增加factor，增加factor的dimension，glm，ANCOVA,  time series model( autocorrelation ) 看是否可以用上。 
</font>
